In [8]:
import requests
import json

In [9]:
#AC_URL = "http://localhost:6380/query"
AC_URL = "https://answercoalesce.renci.org/query"

In [10]:
def generate_trapi_query(input_node_type,output_node_type,input_curies,predicate,input_is_subject=True):
    envelope = {"message":{"query_graph": {"nodes":{"input":{},"output":{}}, "edges":{"edge_0":{}}}}}
    input_node = envelope["message"]["query_graph"]["nodes"]["input"]
    input_node["categories"] = [input_node_type]
    input_node["ids"] = ["uuid:1"]
    input_node["member_ids"] = input_curies
    input_node["set_interpretation"]="MANY"
    output_node = envelope["message"]["query_graph"]["nodes"]["output"]
    output_node["categories"] = [output_node_type]
    query_edge = envelope["message"]["query_graph"]["edges"]["edge_0"]
    if input_is_subject:
        query_edge["subject"] = "input"
        query_edge["object"] = "output"
    else:
        query_edge["subject"] = "output"
        query_edge["object"] = "input"
    query_edge["predicates"]  = [ predicate ]
    #query_edge["knowledge_type"] = "inferred"
    return envelope

In [11]:
def phenotypes_to_disease(phenotypes, print_query=False):
    #Phenotype to Disease
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Disease"
    predicate = "biolink:has_phenotype"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

In [12]:
def genes_to_chemical(genes, print_query=False):
    #Phenotype to Disease
    input_type = "biolink:Gene"
    output_type = "biolink:ChemicalEntity"
    predicate = "biolink:related_to"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, genes, predicate, input_is_subject)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

In [13]:
def phenotypes_to_gene(phenotypes):
    #Phenotype to Gene
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Gene"
    predicate = "biolink:genetically_associated_with"
    input_is_subject = "True"
    query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

In [14]:
def print_results(resp):
    for result in resp["message"]["results"]:
        nb = result["node_bindings"]["output"][0]["id"]
        name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
        edge_id=result["analyses"][0]["edge_bindings"]["edge_0"][0]["id"]
        edge = resp["message"]["knowledge_graph"]["edges"][edge_id]
        p=""
        edge_atts = edge["attributes"]
        for att in edge_atts:
            if att["attribute_type_id"] == "biolink:p_value":
                p = att["value"]
        print(nb, name, p) 

In [15]:
genes = ["NCBIGene:5297", "NCBIGene:5298","NCBIGene:5290"]
r = genes_to_chemical(genes)

ConnectionError: HTTPSConnectionPool(host='answercoalesce.renci.org', port=443): Max retries exceeded with url: /query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x106590380>: Failed to resolve 'answercoalesce.renci.org' ([Errno 8] nodename nor servname provided, or not known)"))

In [98]:
print_results(r)

NameError: name 'r' is not defined

In [64]:
#Leigh Disease?
leigh_phenotypes = ["HP:0000739", # | Anxiety, 
                    "HP:0001288", # | Gait disturbance, 
                    "HP:0001252", # | Hypotonia, 
                    "HP:0001250", # | Seizure, 
                    "HP:0000750", # | Delayed speech and language development, 
                    "HP:0002378", # | Hand tremor, 
                    "HP:0002019", # | Constipation, 
                    "HP:0007146"] # | Bilateral basal ganglia lesions

In [ ]:
input_type = "biolink:PhenotypicFeature"
output_type = "biolink:Disease"
predicate = "biolink:has_phenotype"
input_is_subject = "False"
query = generate_trapi_query( input_type, output_type, leigh_phenotypes, predicate, input_is_subject)
print(json.dumps(query,indent=2))

In [ ]:
response = phenotypes_to_disease(leigh_phenotypes, print_query=True)
print_results(response)

In [67]:
response = phenotypes_to_gene(leigh_phenotypes)
print_results(response)

200
NCBIGene:93986 FOXP2 0.0006474412355706435
NCBIGene:53335 BCL11A 0.0006474412355706435
NCBIGene:85358 SHANK3 0.0008631618162098167
NCBIGene:2077 ERF 0.0008631618162098167
NCBIGene:9378 NRXN1 0.0008631618162098167
NCBIGene:1806 DPYD 0.0010788358313316428
NCBIGene:26047 CNTNAP2 0.0015100442052276759
NCBIGene:26040 SETBP1 0.0015100442052276759
NCBIGene:284058 KANSL1 0.001725578584098671
NCBIGene:50484 RRM2B 0.0019410664376458965
NCBIGene:113189 CHST14 0.0019410664376458965
NCBIGene:2903 GRIN2A 0.002156507775912319
NCBIGene:5728 PTEN 0.003017808176953609
NCBIGene:3737 KCNA2 0.004093388288691872
NCBIGene:3954 LETM1 0.00495301684843197
NCBIGene:4204 MECP2 0.008170020749701742
NCBIGene:18 ABAT 0.011376624022382093


In [68]:
leigh_2 = [ "HP:0001252", #	""Hypotonia"""
            "HP:0002521", #	""Hypsarrhythmia"""
            "HP:0001263", #	""Global developmental delay"""
            "HP:0004305", #	""Involuntary movements"""
            "HP:0002579", #	""Gastrointestinal dysmotility"""
            "HP:0200147", # "Neuronal loss in basal ganglia". 
            "HP:0007183" #	""Focal T2 hyperintense basal ganglia lesion"""
          ]

In [69]:
response = phenotypes_to_disease(leigh_2)
print_results(response)

None
MONDO:0030919 intellectual disability, autosomal dominant 53 5.536048099918041e-11
MONDO:0014942 developmental and epileptic encephalopathy, 45 2.4244693060343317e-10
MONDO:0014842 intellectual disability, autosomal dominant 41 3.849422096046302e-10
MONDO:0014477 developmental and epileptic encephalopathy, 26 1.4929091468683857e-09
MONDO:0011966 periventricular heterotopia with microcephaly, autosomal recessive 1.4929091468683857e-09
MONDO:0013164 beta-ureidopropionase deficiency 1.4929091468683857e-09
MONDO:0030963 Li-Campeau syndrome 1.9379275651635485e-09
MONDO:0033366 developmental and epileptic encephalopathy, 57 1.9379275651635485e-09
MONDO:0010657 methylmalonic acidemia with homocystinuria, type cblX 1.9379275651635485e-09
MONDO:0011964 DPAGT1-congenital disorder of glycosylation 2.4635554288901497e-09
MONDO:0033362 developmental and epileptic encephalopathy, 53 3.076485468182307e-09
MONDO:0011229 ethylmalonic encephalopathy 3.076485468182307e-09
MONDO:0014905 encephalopath

In [70]:
response = phenotypes_to_gene(leigh_2)
print_results(response)

200
NCBIGene:51733 UPB1 0.0001888803437402939
NCBIGene:3295 HSD17B4 0.0003777250116963364
NCBIGene:84152 PPP1R1B 0.0009440450282371355
NCBIGene:284058 KANSL1 0.0015100442052276759
NCBIGene:50484 RRM2B 0.0016986393312994233
NCBIGene:113189 CHST14 0.0016986393312994233
NCBIGene:1356 CP 0.0033943932870210093
NCBIGene:3356 HTR2A 0.0033943932870210093
NCBIGene:7249 TSC2 0.0037708361514733027
NCBIGene:7248 TSC1 0.0037708361514733027
NCBIGene:3954 LETM1 0.004335233855151651
NCBIGene:4204 MECP2 0.0071524297375753
NCBIGene:4988 OPRM1 0.007339959127928182
NCBIGene:1813 DRD2 0.008277074904243129
NCBIGene:1392 CRH 0.008464391871230346
NCBIGene:5173 PDYN 0.008651673457724452
NCBIGene:18 ABAT 0.009961654470562097
NCBIGene:5443 POMC 0.013694926855229318


In [71]:
phenotypes_2 = [
    "HP:0000718", # aggression         | 2 yrs - present
    "HP:0000739", #anxiety                | 2 yrs - present
    "HP:0002616", # Aortic root aneurysm HP:0002616 | 10 yrs - present
    "HP:0002019", #constipation (occasional) HP:0002019 |        birth - present
    #occasional bright lime-green stool | 5 yrs - present
    "HP:0000678", #crowded teeth        HP:0000678         
    "HP:0000886", # Deformed rib cage HP:0000886 | 6 months - present
    "HP:0002046", #heat intolerance        HP:0002046 
    "HP:0002705", #high arched palate        HP:0002705 
    "HP:0001290", #hypotonia        HP:0001290 |        birth - present
    "HP:0000737", # irritable        HP:0000737 |        2 yrs - present
    "HP:0001382", # Joint hypermobility HP:0001382 | birth - present
    "HP:0000720", # mood swings        HP:0000720 |        2 yrs - present
    "HP:0012801", # narrow jaw        HP:0012801
    "HP:0010865", # Oppositional defiant disorder HP:0010865 | 2 yrs - present
    #Destructive tendencies | 3 yrs - present
    "HP:0002172", # Postural instability HP:0002172 | birth - present
    "HP:0001250", # seizures        HP:0001250 |        7 yrs - present
    "HP:0001649", # Tachycardia HP:0001649 | 1 yr - present
    "HP:0001337", # tremor        HP:0001337 | in hands 11 yrs - present
    #Vascular skin abnormality (superficial blood vessels that rupture easily) HP:0011276 |  birth - present
    "HP:0001123" # Visual field defect HP:0001123 | birth - present
    # Seasonal allergies | birth - present
    #Food allergy | 4 yrs - present
]

In [72]:
response = phenotypes_to_disease(phenotypes_2)
print_results(response)

None
MONDO:0010501 syndromic X-linked intellectual disability 34 2.342808145498639e-13
MONDO:0010441 CK syndrome 3.4695918945210576e-12
MONDO:0010306 X-linked intellectual disability, Cabezas type 2.012562565025918e-09
MONDO:0010671 microphthalmia, syndromic 1 2.303903948394455e-09
MONDO:0010655 X-linked intellectual disability with marfanoid habitus 4.259526562078124e-09
MONDO:0010500 intellectual disability, X-linked, syndromic 33 1.763189694158661e-08
MONDO:0032605 intellectual disability, autosomal recessive 66 2.237839556363733e-08
MONDO:0014606 intellectual disability-microcephaly-strabismus-behavioral abnormalities syndrome 2.576201852588367e-08
MONDO:0859143 Radio-Tartaglia syndrome 2.7675996639343947e-08
MONDO:0019349 Sotos syndrome 2.9694267590458545e-08
MONDO:0958179 glycine encephalopathy 1 4.9108665950578294e-08
MONDO:0013933 peroxisome biogenesis disorder 5B 7.538205405614794e-08
MONDO:0008082 multiple endocrine neoplasia type 2B 9.145073485559841e-08
MONDO:0100186 GTP cy

In [73]:
response = phenotypes_to_gene(phenotypes_2)
print_results(response)

200
NCBIGene:7200 TRH 0.00043350422688401063
NCBIGene:6262 RYR2 0.0010249216925187008
NCBIGene:6343 SCT 0.0010249216925187008
NCBIGene:10451 VAV3 0.002048792920561606
NCBIGene:3360 HTR4 0.0025603349468989512
NCBIGene:51083 GAL 0.005624085345056082
NCBIGene:3972 LHB 0.005624085345056082
NCBIGene:2778 GNAS 0.011216540354143694
NCBIGene:2796 GNRH1 0.011216540354143694
NCBIGene:1132 CHRM4 0.011216540354143694
NCBIGene:7432 VIP 0.012736289207362732
NCBIGene:196 AHR 0.01324235290623453
NCBIGene:2641 GCG 0.01324235290623453
NCBIGene:3827 KNG1 0.01475898810876207
NCBIGene:154 ADRB2 0.01475898810876207
NCBIGene:3952 LEP 0.017785268943805686
NCBIGene:43 ACHE 0.020300067645087804
NCBIGene:6334 SCN8A 0.020300067645087804
NCBIGene:1813 DRD2 0.022307269930771232
NCBIGene:1565 CYP2D6 0.022307269930771232
NCBIGene:590 BCHE 0.0253103653519519
NCBIGene:3630 INS 0.026808452157561898
NCBIGene:3350 HTR1A 0.026808452157561898
NCBIGene:135 ADORA2A 0.027307302575163592
NCBIGene:538 ATP7A 0.028304236420908836


In [74]:
tbcd_phenotypes=[
    "HP:0002098", # | Respiratory distress, 
    "HP:0001252", # | Hypotonia, 
    "HP:0001250" # | Seizure
]

In [75]:
response = phenotypes_to_disease(tbcd_phenotypes)
print_results(response)

None
MONDO:0014271 STT3B-congenital disorder of glycosylation 2.096353238263982e-07
MONDO:0014856 combined oxidative phosphorylation defect type 30 5.534189247952736e-07
MONDO:0010490 SSR4-congenital disorder of glycosylation 5.534189247952736e-07
MONDO:0008056 myotonic dystrophy type 1 5.534189247952736e-07
MONDO:0012792 mitochondrial DNA depletion syndrome 8a 7.367208238964271e-07
MONDO:0014754 primary coenzyme Q10 deficiency 8 8.381793480638825e-07
MONDO:0859279 spinal muscular atrophy, distal, autosomal recessive, 6 9.461748430404016e-07
MONDO:0013307 myopathy, lactic acidosis, and sideroblastic anemia 2 1.060706249785143e-06
MONDO:0060704 neurodevelopmental disorder with spastic quadriplegia and brain abnormalities with or without seizures 1.060706249785143e-06
MONDO:0009612 methylmalonic aciduria due to methylmalonyl-CoA mutase deficiency 1.060706249785143e-06
MONDO:0010271 X-linked myotubular myopathy-abnormal genitalia syndrome 1.060706249785143e-06
MONDO:0009614 methylmalonic 

In [76]:
response = phenotypes_to_gene(tbcd_phenotypes)
print_results(response)

200
NCBIGene:284058 KANSL1 0.0006474412355706435
NCBIGene:50484 RRM2B 0.0007283419108450321
NCBIGene:113189 CHST14 0.0007283419108450321
NCBIGene:3954 LETM1 0.001860263944114525
NCBIGene:4204 MECP2 0.003071614760772548
NCBIGene:18 ABAT 0.004281495471845135


In [77]:
wdyf_phenotypes = [
    #Global developmental delay 
    "HP:0001263",
    #Seizure 
    "HP:0001250",
    #Neurodevelopmental delay 
    "HP:0012758",
    #Delayed social development 
    "HP:0012434"
]

In [78]:
response = phenotypes_to_disease(wdyf_phenotypes)
print_results(response)

None
MONDO:0012830 chromosome 10q23 deletion syndrome 7.043541590830241e-07
MONDO:0014992 lissencephaly 8 1.489667817869212e-06
MONDO:0032866 cortical dysplasia, complex, with other brain malformations 10 1.8850895473043174e-06
MONDO:0014908 microcephaly 17, primary, autosomal recessive 2.32693618090683e-06
MONDO:0060761 neurodevelopmental disorder with epilepsy and hypoplasia of the corpus callosum 2.5652625674495586e-06
MONDO:0060758 spinocerebellar ataxia 42, early-onset, severe, with neurodevelopmental deficits 2.8151876576608396e-06
MONDO:0029143 intellectual developmental disorder with hypertelorism and distinctive facies 3.076708944928911e-06
MONDO:0014351 pontocerebellar hypoplasia type 9 3.3498239230479854e-06
MONDO:0033492 Coffin-Siris syndrome 6 4.5581706341248245e-06
MONDO:0014601 autosomal recessive spinocerebellar ataxia 20 5.951816007870529e-06
MONDO:0013076 attention deficit-hyperactivity disorder, susceptibility to, 7 0.00010793599385078916
MONDO:0033670 hearing loss, 

In [ ]:
response = phenotypes_to_gene(wdyf_phenotypes)
print_results(response)

In [ ]:
twogene_phenotypes = [
    #Recurrent fever 
    "HP:0001954",
    #hip dysplasia        
    "HP:0001385",
    #joint hypermobility        
    "HP:0001382",
    #optic disc pallor        
    "HP:0000543",        
    #recurrent infections        
    "HP:0002719",        
    #thin corpus callosum        
    "HP:0002079",
    #Global developmental delay 
    "HP:0001263",
    #Abnormality of vision 
    "HP:0000504",
    #Nystagmus 
    "HP:0000639",
    #Drooling 
    "HP:0002307",
    #Dysphagia 
    "HP:0002015",
    #Hypotonia 
    "HP:0001252"
    #Eczema 
    "HP:0000964",
    #Inability to walk 
    "HP:0002540",
    #Septo-optic dysplasia 
    "HP:0100842"
    #Postural instability 
    "HP:0002172"
]

In [83]:
response = phenotypes_to_disease(twogene_phenotypes)
print_results(response)

None
MONDO:0030990 Kohlschutter-Tonz syndrome-like 3.267366897642553e-13
MONDO:0032781 congenital hypotonia, epilepsy, developmental delay, and digital anomalies 1.5438766550103143e-12
MONDO:0010533 Arts syndrome 1.7537587683174986e-11
MONDO:0010283 syndromic X-linked intellectual disability Lubs type 2.177644306855737e-11
MONDO:0014746 SLC39A8-CDG 2.4148326614321293e-11
MONDO:0033371 developmental and epileptic encephalopathy, 62 2.4148326614321293e-11
MONDO:0014905 encephalopathy due to defective mitochondrial and peroxisomal fission 2 4.2777578280153676e-11
MONDO:0032710 developmental and epileptic encephalopathy, 72 5.536048099918041e-11
MONDO:0014979 myoclonus, intractable, neonatal 7.053170414013553e-11
MONDO:0044701 childhood-onset motor and cognitive regression syndrome with extrapyramidal movement disorder 8.862321093680521e-11
MONDO:0011897 leukoencephalopathy-ataxia-hypodontia-hypomyelination syndrome 1.3499416171165125e-10
MONDO:0033364 developmental and epileptic encephalo

In [84]:
response = phenotypes_to_gene(twogene_phenotypes)
print_results(response)

200
NCBIGene:84466 MEGF10 0.0011327470604279887
NCBIGene:79042 TSEN34 0.0037708361514733027
NCBIGene:283989 TSEN54 0.004147136824040217
NCBIGene:80746 TSEN2 0.0045232953584311885
NCBIGene:6863 TAC1 0.012016699562439063


In [85]:
autism_phenotypes=[
    #Autistic behavior 
    "HP:0000729",
    #Neurodevelopmental delay 
    "HP:0012758",
    #Intellectual disability 
    "HP:0001249",
    #Ventricular septal defect 
    "HP:0001629",
    #Abnormal facial shape 
    "HP:0001999",
    #high arched palate        
    "HP:0002705",
    #high nasal bridge        
    "HP:0000426",
    #shallow orbits        
    "HP:0000586",
    #Abnormality of the palmar creases 
    "HP:0010490"
]

In [86]:
response = phenotypes_to_disease(autism_phenotypes)
print_results(response)

None
MONDO:0033492 Coffin-Siris syndrome 6 5.214647608277274e-08
MONDO:0013256 chromosome 15q24 deletion syndrome 1.7122897278399835e-06
MONDO:0008736 peroxisome biogenesis disorder 2B 4.23870594654196e-06
MONDO:0032850 neurooculocardiogenitourinary syndrome 6.61976266806481e-06
MONDO:0859252 neurodevelopmental disorder with poor growth and skeletal anomalies 1.422372092350377e-05
MONDO:0030030 Nizon-Isidor syndrome 2.3017716165030653e-05
MONDO:0008999 Cohen syndrome 3.592446802548804e-05
MONDO:0030679 Noonan syndrome 14 3.800047203182352e-05
MONDO:0014831 progeroid and marfanoid aspect-lipodystrophy syndrome 3.800047203182352e-05
MONDO:0010664 syndromic X-linked intellectual disability Snyder type 4.457613506478945e-05
MONDO:0009318 Hallermann-Streiff syndrome 5.415381663330505e-05
MONDO:0014700 Au-Kline syndrome 5.928934823615599e-05
MONDO:0011150 acroosteolysis-keloid-like lesions-premature aging syndrome 6.465570624499397e-05
MONDO:0013415 chromosome 17p13.1 deletion syndrome 8.213

In [ ]:
response = phenotypes_to_gene(autism_phenotypes)
print_results(response)